In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9182k      0  0:00:08  0:00:08 --:--:-- 14.6M


In [2]:
!rm -r aclImdb/train/unsup

In [3]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  # Shuffle the list of training files using a seed, to ensure we get the same validation set every time we run the code.
  random.Random(1337).shuffle(files)
  # take 20% files of the training files to use for validation.
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  # move the files to aclImdb/val/neg and aclImdb/val/pos
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [5]:
!cat aclImdb/val/pos/10010_7.txt

There are many illnesses born in the mind of man which have been given life in modern times. Constant vigilance or accrued information in the realm of Pyschosis, have kept psychologists, counselors and psychiatrists busy with enough work to last them decades. Occasionally, some of these mental phenomenon are discover by those with no knowledge of their remedy or even of their existence. That is the premise of the film entitled " The Night Listner." It tells the story of a popular radio host called Gabriel Noon (Robin Williams) who spends his evenings enthralling his audiences with vivid stories about Gay lifestyles. Perhaps its because his show is losing it's authentic veneer which causes Noon to admit he is no longer himself. Feeling abandoned by both his lover Jess (Bobby Cannavale) and his and best friend (Joe Morton), he seeks shelter in his deepening despair and isolation. It is here, a mysterious voice in the night asks him for help. Noon needs to feel useful and reaches out to t

In [6]:
from tensorflow import keras
batch_size = 32

# Each subfolder name becomes a class label.
# Keras automatically sorts folder names alphabetically to determine numeric labels

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size = batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size = batch_size
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [7]:
for inputs, targets in train_ds:
  print("inputs.shape", inputs.shape)
  print("inputs.dtype", inputs.dtype)
  print("targets.shape", targets.shape)
  print("targets.dtype", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape (32,)
inputs.dtype <dtype: 'string'>
targets.shape (32,)
targets.dtype <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Savage Island (2003) is a lame movie. It\'s more like a home video shot with very minimal lighting and horrid acting. Not only that the storyline and script was wretched. I don\'t know why this movie was made. I have seen a lot of flicks in my time and the ones I really hate are movies that make me angry. This one made my blood boil. The situations were inane at best. If I made a movie like this it would have been a short. Really because those backwood "idjits" wouldn\'t have been in the picture.<br /><br />Don\'t be fooled by the cover on the D.V.D. I am an avid watcher of bad cinema. But this movie is virtually unwatchable. I don\'t mind movies being shot on D.V. but if you\'re going to do that make the movie enjoyable, not some tired retread of superior horror films (sans Wrong Turn).<br /><br />I have to not recommend this waste of disk. If you come across thi

In [8]:
# Processing words as a set: The bag-of-words approach
# Single words (UNIGRAMS) with binary encoding

from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens = 20000,
    output_mode = "multi_hot", # one vector per sentence
    # one_hot = one vector per word
    # int = one integer per word
)

text_only_train_ds = train_ds.map(lambda x, y:x) # yields only raw text as input (not label)
text_vectorization.adapt(text_only_train_ds) # use that dataset to index the dataset vocabulary via the adapt() method

vocab = text_vectorization.get_vocabulary()
print(vocab[:50])
print()

inverse_vocab = {word: idx for idx, word in enumerate(vocab)}

for k, v in list(inverse_vocab.items())[:10]:
    print(k, v)



['[UNK]', np.str_('the'), np.str_('a'), np.str_('and'), np.str_('of'), np.str_('to'), np.str_('is'), np.str_('in'), np.str_('it'), np.str_('i'), np.str_('this'), np.str_('that'), np.str_('br'), np.str_('was'), np.str_('as'), np.str_('with'), np.str_('for'), np.str_('but'), np.str_('movie'), np.str_('film'), np.str_('on'), np.str_('not'), np.str_('you'), np.str_('his'), np.str_('are'), np.str_('have'), np.str_('he'), np.str_('be'), np.str_('one'), np.str_('its'), np.str_('at'), np.str_('all'), np.str_('by'), np.str_('an'), np.str_('they'), np.str_('who'), np.str_('from'), np.str_('so'), np.str_('like'), np.str_('her'), np.str_('or'), np.str_('just'), np.str_('about'), np.str_('has'), np.str_('if'), np.str_('out'), np.str_('some'), np.str_('there'), np.str_('what'), np.str_('good')]

[UNK] 0
the 1
a 2
and 3
of 4
to 5
is 6
in 7
it 8
i 9


In [9]:
binary_1gram_train_ds = train_ds.map(
    # converts each review into a 20,000-dimensional multi-hot vector
    # lambda arguments : expression
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

In [10]:
for inputs, targets in binary_1gram_train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", inputs.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int64'>
inputs[0]: tf.Tensor([1 1 1 ... 0 0 0], shape=(20000,), dtype=int64)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [12]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens = 20000, hidden_dim = 16):
  inputs = keras.Input(shape = (max_tokens, ))
  x = layers.Dense(hidden_dim, activation = "relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation = "sigmoid")(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer = "rmsprop",
                loss = "binary_crossentropy",
                metrics = ["accuracy"])
  return model

In [13]:
model = get_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
print(20000*16)

320000


In [15]:
callbacks =  [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only = True)
]

model.fit(binary_1gram_train_ds.cache(),
          # cache() on the datasets to cache them in memory
          # this way, preprocessing only once, during the first epoch, and we'll reuse the preprocessed texts for the following epochs
          # can only be done if the data is small enough to fit in memory
          validation_data = binary_1gram_val_ds.cache(),
          epochs = 10,
          callbacks = callbacks)

model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}") # 88.7%
# random baseline: 50%

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.7673 - loss: 0.5000 - val_accuracy: 0.8930 - val_loss: 0.2734
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8966 - loss: 0.2847 - val_accuracy: 0.8978 - val_loss: 0.2650
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9152 - loss: 0.2457 - val_accuracy: 0.8974 - val_loss: 0.2799
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9206 - loss: 0.2288 - val_accuracy: 0.8952 - val_loss: 0.2883
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.9255 - loss: 0.2205 - val_accuracy: 0.8960 - val_loss: 0.3000
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9278 - loss: 0.2186 - val_accuracy: 0.8962 - val_loss: 0.3133
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9325 - loss: 0.2131 - val_accuracy: 0.8922 - val_loss: 0.3232
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9351 - loss: 0.2126 - val_accur

In [19]:
# BIGRAMS WITH BINARY ENCODING

text_vectorization = TextVectorization(
    ngrams = 2, # local order preservation
    max_tokens = 20000,
    output_mode = "multi_hot",
)

text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls = 4)

binary_2gram_val_ds = val_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)

binary_2gram_test_ds = test_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only = True)
]

model.fit(binary_2gram_train_ds.cache(),
          validation_data = binary_2gram_val_ds.cache(),
          epochs = 10,
          callbacks = callbacks)

model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}") # Test accuracy: 89.8%

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.7670 - loss: 0.4917 - val_accuracy: 0.8986 - val_loss: 0.2716
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9056 - loss: 0.2693 - val_accuracy: 0.9070 - val_loss: 0.2522
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9302 - loss: 0.2151 - val_accuracy: 0.9050 - val_loss: 0.2662
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9379 - loss: 0.2026 - val_accuracy: 0.9050 - val_loss: 0.2780
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9462 - loss: 0.1782 - val_accuracy: 0.9024 - val_loss: 0.3037
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9500 - loss: 0.1673 - val_accuracy: 0.9008 - val_loss: 0.3183
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9540 - loss: 0.1777 - val_accuracy: 0.8984 - val_loss: 0.3225
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9562 - loss: 0.1609 - val_acc

In [22]:
# BIGRAMS WITH TF-IDF ENCODING
# Term frequency, inverse document frequency
"""
def tfidf(term, document, dataset):
  term_freq = document.count(term)

  # raw counts can be very large and uneven. Taking the log:
  # smooths very large values
  # reduces the impact of extreme frequencies

  doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
  return term_freq / doc_freq
"""


text_vectorization = TextVectorization(
    ngrams = 2,
    max_tokens = 20000,
    output_mode = "tf_idf",
)


text_vectorization.adapt(text_only_train_ds)


tfidf_2gram_train_ds = train_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)

tfidf_2gram_val_ds = val_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)

tfidf_2gram_test_ds = test_ds.map(
    lambda x,y: (text_vectorization(x), y),
    num_parallel_calls = 4)


model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only = True)
]

model.fit(tfidf_2gram_train_ds.cache(),
          validation_data = tfidf_2gram_val_ds.cache(),
          epochs = 10,
          callbacks = callbacks)

model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}") # Test accuracy: 87.9%

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.6693 - loss: 0.7452 - val_accuracy: 0.8496 - val_loss: 0.3553
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8006 - loss: 0.3997 - val_accuracy: 0.8774 - val_loss: 0.2939
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.8357 - loss: 0.3540 - val_accuracy: 0.8840 - val_loss: 0.2910
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8473 - loss: 0.3391 - val_accuracy: 0.8860 - val_loss: 0.2857
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8566 - loss: 0.3360 - val_accuracy: 0.8732 - val_loss: 0.2973
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8648 - loss: 0.3230 - val_accuracy: 0.8446 - val_loss: 0.3995
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8733 - loss: 0.3027 - val_accuracy: 0.8808 - val_loss: 0.3097
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8809 - loss: 0.2966 - val_accuracy

In [23]:
inputs = keras.Input(shape = (1,), dtype = "string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."]
])

predictions = inference_model(raw_text_data)
print(f"{float(predictions[0]*100):.2f} percent positive")

95.94 percent positive
